In [1]:
import io
from typing import Dict, List
from datetime import datetime, timedelta
from pydantic import BaseModel
from time import sleep

In [2]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
import sqlalchemy

In [4]:
from data_structures import GameModel, Clock, MoveAnalysis, Opening, Players, Player, GlobalAnalysis, User
from chessportals_communication import lichess_communication

## Functions

In [5]:
# Read PGN string into python-chess object
def read_pgn(game_str):
    pgn = io.StringIO(game_str)
    game = chess.pgn.read_game(pgn)
    return game

In [6]:
# Read python-chess object, play and display it in them on a board.
def play_game(game):
    board = game.board()
    for move in game.mainline_moves():
        clear_output(wait=True)
        print(board)
        board.push(move)
        sleep(1)

In [7]:
# Function that returns list of position evaluations for entire game.
# Scores are in centipawn and from White perspective
# If value is None it's because engine saw checkmate in #n
def read_evals(game):
    
    eval_lst = []
    for node in game.mainline():
        if node.eval() != None:
            eval_lst.append(node.eval().white().score())

    return eval_lst

def read_moves(game):
    
    move_lst = [str(node.san()) for node in game.mainline()]
    return(move_lst)

def read_game_eval_recursive(game_node):
    
    if game_node.is_end():
        print(game_node.eval())
        return [None]
    else:
        if game_node.eval()!=None:
            return [game_node.eval().white().score()] + read_game_eval(game_node[0])
        else:
            return read_game_eval(game_node[0])   

## Fetch games from Lichess API into DataFrame

In [8]:
lichess_id = 'miguel0f'

last_Xdays = datetime.now()-timedelta(days=20)
since = last_Xdays
until = datetime.now()

lichess_comm = lichess_communication(lichess_id)

In [9]:
lichess_comm.fetch_user_data()
lichess_comm.show_user_data()

User miguel0f created at 2020-12-05 15:38:45.080000+00:00
Rating Classical : 1705 (  58 games)
Rating Rapid     : 1692 ( 907 games)
Rating Blitz     : 1251 (1143 games)
Rating Bullet    :  989 (  24 games)


In [10]:
lichess_comm.fetch_games_by_dates(since,until)
lichess_comm.fill_df()

[MoveAnalysis(mate='nan', eval=0, best=None, variation=None, judgment=''), MoveAnalysis(mate='nan', eval=174, best='g8f6', variation='Nf6 c4 e6 Nf3 d5 cxd5 exd5 Bf4 c6 Nc3', judgment=Judgment(name='Blunder', comment='Blunder. Nf6 was best.')), MoveAnalysis(mate='nan', eval=144, best=None, variation=None, judgment=''), MoveAnalysis(mate='nan', eval=146, best=None, variation=None, judgment=''), MoveAnalysis(mate='nan', eval=117, best=None, variation=None, judgment=''), MoveAnalysis(mate='nan', eval=162, best=None, variation=None, judgment=''), MoveAnalysis(mate='nan', eval=96, best='b1c3', variation='Nc3 Nf6 Nb5 Nc6 Nxd6+ cxd6 e3 Ne4 Ne2 Bg4', judgment=Judgment(name='Inaccuracy', comment='Inaccuracy. Nc3 was best.')), MoveAnalysis(mate='nan', eval=90, best=None, variation=None, judgment=''), MoveAnalysis(mate='nan', eval=82, best=None, variation=None, judgment=''), MoveAnalysis(mate='nan', eval=84, best=None, variation=None, judgment=''), MoveAnalysis(mate='nan', eval=125, best=None, var

,id,color,opponent,time_control,datetime,opening,result,moves,analysis,evals,mates,judgment
0,3K7E0KBD,black,pionnenvreter,classical,2022-07-22 22:34:57.139000+00:00,Englund Gambit Complex: Hartlaub-Charlick Gambit,loss,"[d4, e5, dxe5, d6, exd6, Bxd6, c3, Nc6, Nf3, Q...",True,"[0, 174, 144, 146, 117, 162, 96, 90, 82, 84, 1...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[, Blunder, , , , , Inaccuracy, , , , , , , , ..."
1,54KlyM6F,black,heaveight,classical,2022-07-22 21:37:13.352000+00:00,Elephant Gambit: Paulsen Countergambit,loss,"[e4, e5, Nf3, d5, exd5, e4, Qe2, Nf6, Nc3, Bb4...",True,"[33, 12, 19, 123, 130, 140, 136, 119, 95, 211,...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[, , , Inaccuracy, , , , , , Inaccuracy, , , ,..."
2,btz02tRd,black,riku_g,classical,2022-07-21 19:58:30.434000+00:00,"Scotch Game: Scotch Gambit, London Defense",loss,"[e4, e5, Nf3, Nc6, d4, exd4, Bc4, Bb4+, c3, dx...",True,"[33, 12, 19, 32, 13, 13, 0, -13, 0, 0, 0, -25,...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[, , , , , , , , , , , , , Inaccuracy, , , Ina..."
3,mDMXMiw3,black,dompart519,classical,2022-07-21 13:08:47.216000+00:00,"Caro-Kann Defense: Advance Variation, Bayonet ...",loss,"[e4, c6, d4, d5, e5, Bf5, g4, Bg6, e6, fxe6, h...",True,"[33, 56, 52, 28, 28, 50, 0, 44, 18, 9, 0, 133,...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[, , , , , , , , , , , Mistake, , , , , Inaccu..."
4,zCZx0KC7,black,shemon11,classical,2022-07-21 10:32:47.533000+00:00,Scotch Game,win,"[e4, e5, Nf3, Nc6, d4, Bd6, Bc4, f6, dxe5, fxe...",True,"[33, 12, 19, 32, 13, 125, 50, 288, 282, 521, 4...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[, , , , , Mistake, Inaccuracy, Blunder, , Mis..."
5,Vzn667SP,black,branovasil,classical,2022-07-20 20:09:19.264000+00:00,"Scotch Game: Scotch Gambit, London Defense",win,"[e4, e5, Nf3, Nc6, d4, exd4, Bc4, Bb4+, c3, dx...",True,"[33, 12, 19, 32, 13, 13, 0, -13, 0, 0, 0, 99, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[, , , , , , , , , , , Inaccuracy, , , , Blund..."
6,My7vAPF8,black,branovasil,classical,2022-07-20 19:43:35.991000+00:00,Englund Gambit Complex: Hartlaub-Charlick Gambit,win,"[d4, e5, dxe5, d6, exd6, Bxd6, c3, Nc6, e4, Qe...",True,"[0, 174, 144, 146, 117, 162, 96, 90, 0, 1, -13...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[, Blunder, , , , , Inaccuracy, , Inaccuracy, ..."
7,jKKSsOq4,black,aleksanderdaniloff,blitz,2022-07-20 09:46:06.365000+00:00,Elephant Gambit,win,"[e4, e5, Nf3, d5, Nxe5, dxe4, Nc3, Nf6, d3, Bb...",True,"[33, 12, 19, 123, 76, 82, -57, 89, -19, 0, -48...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[, , , Inaccuracy, , , Mistake, Mistake, Inacc..."
8,6hxDZllQ,black,oknekim,blitz,2022-07-19 15:55:34.679000+00:00,Elephant Gambit: Paulsen Countergambit,loss,"[e4, e5, Nf3, d5, exd5, e4, Ng1, Qxd5, Nc3, Qe...",True,"[33, 12, 19, 123, 130, 140, 0, 5, 13, 35, 9, 5...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[, , , Inaccuracy, , , Mistake, , , , , , , , ..."
9,Bum7UaeP,black,uos,blitz,2022-07-19 11:31:25.303000+00:00,Russian Game: Stafford Gambit,loss,"[e4, e5, Nf3, Nf6, Nxe5, Nc6, Nxc6, dxc6, Nc3,...",False,None,None,None


## Save into Database

In [11]:
from sqlalchemy import create_engine, select, MetaData, Table, Column, Integer, String, DateTime, Boolean
from sqlalchemy.orm import declarative_base, Session
from sqlalchemy_utils import ScalarListType

Base = declarative_base()

dtypes = {'id': String,
          'color': String,
          'opponent': String,
          'time_control': String,
          'datetime': DateTime,
          'opening': String,
          'result': String,
          'moves': ScalarListType(),
          'analysis': Boolean,
          'evals': ScalarListType(float),
          'mates': ScalarListType(float),
          'judgment': ScalarListType()
}

class User(Base):
    __tablename__ = 'user_account'

    id = Column(Integer, primary_key=True)
    name = Column(String)

    def __repr__(self):
        return f'<User(name={self.name})>'

class Games(Base):
    __tablename__ = 'games'
    
    id = Column(dtypes['id'], primary_key=True)
    color = Column(dtypes['color'])
    opponent = Column(dtypes['opponent'])
    time_control = Column(dtypes['time_control'])
    datetime = Column(dtypes['datetime'])
    opening = Column(dtypes['opening'])
    result = Column(dtypes['result'])
    moves = Column(dtypes['moves'])
    analysis = Column(dtypes['analysis'])
    evals = Column(dtypes['evals'])
    mates = Column(dtypes['mates'])
    judgment = Column(dtypes['judgment'])
    
    def __repr__(self):
        return f'<Game(id={self.id}, color={self.color}, result={self.result}, opening={self.opening})>'
    
engine = create_engine('sqlite:///:memory:', echo=True, future=True)
Base.metadata.create_all(engine)


session = Session()
    
lichess_comm.games_df.to_sql('games', 
                             engine, 
                             if_exists='append',
                             dtype=dtypes,
                             index=False,
                             index_label='id')

2022-07-23 03:37:21,924 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-23 03:37:21,926 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-07-23 03:37:21,927 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-23 03:37:21,930 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-07-23 03:37:21,932 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-23 03:37:21,934 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("games")
2022-07-23 03:37:21,935 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-23 03:37:21,937 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("games")
2022-07-23 03:37:21,938 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-23 03:37:21,941 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	PRIMARY KEY (id)
)


2022-07-23 03:37:21,942 INFO sqlalchemy.engine.Engine [no key 0.00116s] ()
2022-07-23 03:37:21,949 INFO sqlalchemy.engine.Engine 
CREATE TABLE games (
	id VARC

2022-07-23 03:37:21,988 INFO sqlalchemy.engine.Engine COMMIT


41

In [12]:
stmt = select(Games).where(Games.color == 'black')

with Session(engine) as session:
    
    rows = session.query(Games).count()
    print(f'** Number of rows : {rows} **')
        
    results = session.execute(stmt).fetchall()
    for result in results:
        print(result[0])

2022-07-23 03:37:22,012 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-23 03:37:22,027 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT games.id AS games_id, games.color AS games_color, games.opponent AS games_opponent, games.time_control AS games_time_control, games.datetime AS games_datetime, games.opening AS games_opening, games.result AS games_result, games.moves AS games_moves, games.analysis AS games_analysis, games.evals AS games_evals, games.mates AS games_mates, games.judgment AS games_judgment 
FROM games) AS anon_1
2022-07-23 03:37:22,032 INFO sqlalchemy.engine.Engine [generated in 0.00433s] ()
** Number of rows : 41 **
2022-07-23 03:37:22,037 INFO sqlalchemy.engine.Engine SELECT games.id, games.color, games.opponent, games.time_control, games.datetime, games.opening, games.result, games.moves, games.analysis, games.evals, games.mates, games.judgment 
FROM games 
WHERE games.color = ?
2022-07-23 03:37:22,041 INFO sqlalchemy.engine.Engine [genera

In [13]:
for k in range(len(lichess_comm.games_lst)):
    opponent = lichess_comm.games_lst[k].players.black.user.id
    if opponent == 'man05060135':
        print(lichess_comm.games_lst[k])

id='yupZEyr2' rated=True variant='standard' speed='rapid' createdAt=datetime.datetime(2022, 7, 18, 23, 4, 35, 34000, tzinfo=datetime.timezone.utc) lastMoveAt=datetime.datetime(2022, 7, 18, 23, 46, 55, 91000, tzinfo=datetime.timezone.utc) status='draw' players=Players(white=Player(user=User(name='miguel0f', id='miguel0f'), rating=1693, ratingDiff=-1, analysis=GlobalAnalysis(inaccuracy=10, mistake=3, blunder=4, acpl=55)), black=Player(user=User(name='Man05060135', id='man05060135'), rating=1672, ratingDiff=1, analysis=GlobalAnalysis(inaccuracy=4, mistake=4, blunder=6, acpl=57))) winner='draw' opening=Opening(eco='C44', name='Scotch Game', ply=5) moves='e4 e5 Nf3 Nc6 d4 d5 Nxe5 dxe4 Nxc6 bxc6 Bc4 c5 Bb5+ Ke7 d5 a6 Bc6 Rb8 O-O Bb7 Re1 f5 Nc3 Bxc6 dxc6 Qxd1 Rxd1 Nf6 Bf4 h6 Bxc7 Rc8 Bd6+ Kf7 c7 Bxd6 Rxd6 Rxc7 Rxa6 Rb7 b3 Rhb8 Rd1 Ne8 Kf1 Kf8 Rd5 Nc7 Rxf5+ Kg8 Ra5 Ne6 Re5 Nd4 Nxe4 Nxc2 Raxc5 Nd4 f4 Rf8 g3 Rd7 b4 Nf3 Kg2 Nxe5 Rxe5 Rdd8 a4 Rd4 b5 Rxa4 b6 Ra2+ Kh3 Rb8 Re6 Rb2 f5 R2xb6 Re7 Kf8 Rd

In [14]:
for k in range(len(lichess_comm.games_lst)):
    speed = lichess_comm.games_lst[k].speed
    pref = lichess_comm.games_lst[k].perf
    if speed !=  pref:
        print(speed, pref)

AttributeError: 'GameModel' object has no attribute 'perf'

In [ ]:
print(type(lichess_comm.games_lst[0]))
[move.eval for move in lichess_comm.games_lst[0].analysis]

In [ ]:
len(lichess_comm.games_lst)

In [ ]:
games_df['result'].value_counts()

In [ ]:
games_df.loc[games_df['result']=='loss']

In [ ]:
game = read_pgn(games_lst[12])
game_eval = games_df.iloc[12]
play_game(game)

plt.figure()
plt.plot(game_eval)
plt.show()

## Code snippets to look into data returned by API

In [ ]:
for k in range(len(a)):
    if a[k]['players'][color]['user']['name']!='miguel0f':
        print('ahahah')